In [187]:
from __future__ import annotations
import typing
from dataclasses import dataclass
import warnings
from contextlib import contextmanager

In [188]:
import numpy as np

In [189]:
@contextmanager
def localize_globals(*exceptions: str, restore_values: bool = True):
    exceptions: typing.Set[str] = set(exceptions)

    old_globals: typing.Dict[str, typing.Any] = dict(globals())
    allowed: typing.Set[str] = set(old_globals.keys())
    allowed.update(exceptions)

    yield None

    new_globals: typing.Dict[str, typing.Any] = globals()

    for name in tuple(new_globals.keys()):
        if name not in allowed:
            del new_globals[name]
    
    if not restore_values:
        return
    
    new_globals.update(
        {k: v for k, v in old_globals.items() if k not in exceptions}
    )

In [190]:
@dataclass(frozen=True)
class Equation:
    A: np.ndarray
    b: np.ndarray
    
    @staticmethod
    def random(size: int) -> Equation:
        A: np.ndarray = np.random.rand(size, size)
        b: np.ndarray = np.random.rand(size)
        
        return Equation(A, b)
    
    @staticmethod
    def random_many(size: int, *, count: int) -> typing.Iterable[Equation]:
        return (Equation.random(size) for _ in range(count))

In [191]:
@dataclass(frozen=True)
class Solution:
    L: np.ndarray
    U: np.ndarray
    Q: np.ndarray
    x: np.ndarray

In [192]:
def matrix_swap(arr: np.ndarray, i: int, j: int,
                axis: int = 0) -> None:
    if i == j:
        return
    
    # Doesn't affect the contents
    arr = arr.swapaxes(0, axis)
    
    arr[[i, j]] = arr[[j, i]]

In [193]:
def solve_gauss(equation: Equation) -> Solution:
    A: np.ndarray = equation.A
    b: np.ndarray = equation.b
    
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    
    size: int = A.shape[0]
    
    L: np.ndarray = np.eye(size)
    U: np.ndarray = A.copy()
    Q: np.ndarray = np.eye(size)
    
    for row_idx in range(size):
        pivot: int = np.argmax(np.abs(U[row_idx:, row_idx])) + row_idx
        
        matrix_swap(U, row_idx, pivot, axis=1)
        matrix_swap(Q, row_idx, pivot, axis=1)
        
        normalization_coeff: float = U[row_idx, row_idx]
        if np.abs(normalization_coeff) < 1e-10:
            warnings.warn("Matrix is close to singular")
        
        # L_cur: np.ndarray = np.eye(size)
        
        for other_row_idx in range(row_idx + 1, size):
            row_coeff: float = U[other_row_idx, row_idx] / normalization_coeff
            U[other_row_idx] -= U[row_idx] * row_coeff
            L[other_row_idx, row_idx] += row_coeff
        # L = L @ L_cur
    
    # Now A @ Q = L @ U
    # L @ U @ Q.T @ x = b
    
    result_l: np.ndarray = solve_triangular(Equation(L, b), lower=True)
    result_u: np.ndarray = solve_triangular(Equation(U, result_l), lower=False)
    x: np.ndarray = Q @ result_u
    
    return Solution(L, U, Q, x)


def solve_triangular(equation: Equation, lower: bool = False) -> np.ndarray:
    A: np.ndarray = equation.A
    b: np.ndarray = equation.b
    
    assert len(A.shape) == 2
    assert len(b.shape) == 1
    assert A.shape[0] == A.shape[1] == b.shape[0]
    
    size: int = A.shape[0]
    
    x: np.ndarray = np.zeros(size)
    
    for row_idx in range(size):
        if not lower:
            row_idx = size - 1 - row_idx
        
        coeff: float = A[row_idx, row_idx]
        if np.abs(coeff) < 1e-10:
            warnings.warn("Matrix is close to singular")
        
        x[row_idx] = (b[row_idx] - A[row_idx] @ x) / coeff
        
    return x

In [194]:
np.random.seed(42)
equation: Equation = Equation.random(4)

In [195]:
true_solution: np.ndarray = np.linalg.solve(equation.A, equation.b)

In [196]:
solution: Solution = solve_gauss(equation)
solution

Solution(L=array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 1.44686189,  1.        ,  0.        ,  0.        ],
       [ 1.62013882,  0.68239209,  1.        ,  0.        ],
       [ 0.30635916,  0.06471274, -0.04634386,  1.        ]]), U=array([[ 5.98658484e-01,  9.50714306e-01,  7.31993942e-01,
         3.74540119e-01],
       [ 0.00000000e+00, -1.21955778e+00, -1.00101053e+00,
        -3.85889184e-01],
       [ 0.00000000e+00,  0.00000000e+00, -4.82265645e-01,
         2.57635751e-01],
       [ 2.77555756e-17,  0.00000000e+00,  0.00000000e+00,
         7.54610627e-01]]), Q=array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.]]), x=array([ 0.24787603, -0.45843549,  0.37849348,  0.61836418]))

In [197]:
assert np.allclose(equation.A @ solution.Q - solution.L @ solution.U, 0)

In [198]:
assert np.allclose(solution.x, true_solution)

In [199]:
def compare_on(equation: Equation) -> float:
    my_result: np.ndarray = solve_gauss(equation).x
    np_result: np.ndarray = np.linalg.solve(equation.A, equation.b)
    
    return np.linalg.norm(my_result - np_result)

In [200]:
def compare_on_random(size: int, count: int) -> None:
    print(f"{size}x{size}:")
    
    for equation in Equation.random_many(size, count=count):
        print("  ", compare_on(equation), sep="")


In [201]:
compare_on_random(3, 10)

3x3:
  4.577566798522237e-16
  5.259223848564703e-15
  8.025154038416283e-16
  2.886579864025407e-15
  8.582937747229195e-17
  0.0
  2.5121479338940403e-15
  5.095246377785861e-16
  4.440892098500626e-16
  3.7238012298709097e-16


In [202]:
compare_on_random(4, 10)

4x4:
  2.791609379366768e-15
  6.431871827503859e-16
  3.967324346521807e-16
  9.075351722425487e-15
  3.1325417310403954e-15
  8.386594157882081e-16
  4.294138290686637e-15
  3.656606164041786e-16
  2.830524433501838e-16
  3.0531133177191805e-16


In [203]:
compare_on_random(5, 10)

5x5:
  3.3009303617866928e-15
  1.852411774394327e-15
  1.0706603116298291e-15
  2.488113585276671e-15
  1.322209902203959e-13
  1.4127342602942564e-14
  8.440616121040728e-15
  1.2964438924174537e-14
  1.9190628093955005e-13
  3.499844917464604e-16
